In [ ]:
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.SaltRemover import SaltRemover


In [ ]:
try:
    from rdkit.Chem.MolStandardize import rdMolStandardize
    from rdkit.Chem.SaltRemover import SaltRemover
    print("✅ MolStandardize is available")
except Exception as e:
    import sys, rdkit
    print("❌ MolStandardize not available")
    print("RDKit version:", rdkit.__version__)
    print("Python:", sys.version)


In [ ]:
import re
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.SaltRemover import SaltRemover

In [ ]:
# === Load NORMAN CSV ===
INPUT_CSV = "susdat_2025-09-07-201212_filtered.csv"
df = pd.read_csv(INPUT_CSV, dtype=str)   # keep everything as text
df = df.dropna(subset=["SMILES"])        # make sure SMILES column exists
all_entries = df.to_dict(orient="records")  # list of dicts (each row = entry)

# === RDKit processing ===
processed = []
unique_smiles = set()
remover = SaltRemover()
uncharger = rdMolStandardize.Uncharger()
normalizer = rdMolStandardize.Normalizer()
# tautomer_canon = rdMolStandardize.TautomerCanonicalizer()

for entry in all_entries:
    smiles = entry["SMILES"]
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue

        # Remove salts/solvents
        mol = remover.StripMol(mol, dontRemoveEverything=True)

        # Handle mixtures → keep largest fragment by heavy atoms
        frags = Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=True)
        if len(frags) > 1:
            mol = max(frags, key=lambda m: m.GetNumHeavyAtoms())

        # Normalize and uncharge
        mol = normalizer.normalize(mol)
        mol = uncharger.uncharge(mol)
        # mol = tautomer_canon.canonicalize(mol)

        # Molecular weight filtering
        mw = rdMolDescriptors.CalcExactMolWt(mol)
        if not (100 <= mw <= 1000):
            continue

        # Canonical SMILES
        can_smiles = Chem.MolToSmiles(mol, canonical=True)
        if can_smiles in unique_smiles:
            continue
        unique_smiles.add(can_smiles)

        # Use existing Formula/ExactMass if present, otherwise compute
        if not entry.get("Formula"):
            entry["Formula"] = rdMolDescriptors.CalcMolFormula(mol)
        if not entry.get("ExactMass"):
            entry["ExactMass"] = mw

        # Add curated SMILES
        entry["Canonical_SMILES"] = can_smiles
        processed.append(entry)

    except Exception:
        continue

# === Save final output ===
cleaned_df = pd.DataFrame(processed)
before_dedup = len(cleaned_df)
cleaned_df = cleaned_df.drop_duplicates(subset="Canonical_SMILES", keep="first")
after_dedup = len(cleaned_df)

cleaned_df.to_csv("Curated_Suspect_List.csv", index=False)

print(f"✅ Done. Final suspect list contains {after_dedup} unique entries.")
print(f"🧹 Duplicates removed after processing: {before_dedup - after_dedup}")


In [ ]:
import pandas as pd

# === Load curated suspect list ===
curated = pd.read_csv("Curated_Suspect_List.csv", dtype=str)

def pick(colnames, candidates):
    for c in candidates:
        if c in colnames: return c
    return None

def dup_report(frame: pd.DataFrame, label: str):
    print(f"\n=== Duplicate report: {label} ===")
    cols = frame.columns

    name_col   = pick(cols, ["Name","Substance","CompoundName","Compound"])
    smiles_col = pick(cols, ["SMILES","Canonical_SMILES","Smiles"])
    can_smi    = "Canonical_SMILES" if "Canonical_SMILES" in cols else None
    inchikey   = pick(cols, ["StdInChIKey","InChIKey","InchiKey","STDINCHIKEY"])
    inchi      = pick(cols, ["InChI","StdInChI","INCHI","STDINCHI"])

    # InChIKey first block (IK14)
    ik14 = None
    if inchikey:
        ik14 = frame[inchikey].astype(str).str.split("-").str[0]

    # normalized name for duplicate check
    name_norm = frame[name_col].astype(str).str.strip().str.lower() if name_col else None

    def count_dups(series, desc):
        s = series.dropna().astype(str)
        n_total = len(series)
        n_nonempty = s.ne("").sum()
        n_unique = s.replace("", pd.NA).dropna().nunique()
        n_dups = n_nonempty - n_unique
        print(f"{desc:>20}: total={n_total}, non-empty={n_nonempty}, unique={n_unique}, duplicates={n_dups}")

        # show top repeated values
        if n_dups > 0:
            vc = s[s.ne("")].value_counts()
            top = vc[vc>1].head(10)
            if not top.empty:
                print(f"Top duplicates for {desc}:")
                display(pd.DataFrame({"count": top}))

    if name_col:
        count_dups(frame[name_col], "Name (raw)")
        count_dups(name_norm,       "Name (normalized)")
    if smiles_col:
        count_dups(frame[smiles_col], "SMILES")
    if can_smi and can_smi != smiles_col:
        count_dups(frame[can_smi], "Canonical_SMILES")
    if inchikey:
        count_dups(frame[inchikey], "InChIKey (full)")
    if ik14 is not None:
        count_dups(ik14, "InChIKey IK14")
    if inchi:
        count_dups(frame[inchi], "InChI (full)")

# Run on curated file
dup_report(curated, "Curated_Suspect_List.csv")
